In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0


In [2]:
from google.colab import files

def upload_and_load_data():
    """Upload CSV file and load GNSS data"""
    print("Please upload your GNSS CSV file:")
    uploaded = files.upload()

    # Get the uploaded filename
    filename = list(uploaded.keys())[0]

    # Load the dataset
    data = pd.read_csv(filename)
    print(f"Dataset shape: {data.shape}")
    print("Dataset columns:", data.columns.tolist())
    print("\nFirst few rows:")
    print(data.head())

    return data

# Load data
data = upload_and_load_data()


Please upload your GNSS CSV file:


Saving generated_data.csv to generated_data.csv
Dataset shape: (672, 5)
Dataset columns: ['epoch', 'x', 'y', 'z', 'clockBias_x']

First few rows:
                      epoch             x             y             z  \
0  2025-01-04T21:29:25.463Z  2.640057e+07  2.649842e+07  2.655433e+07   
1  2025-01-07T22:24:37.502Z  2.655669e+07  2.641471e+07  2.655165e+07   
2  2025-01-07T07:32:21.418Z  2.643634e+07  2.642623e+07  2.665447e+07   
3  2025-01-03T16:21:50.336Z  2.668143e+07  2.642483e+07  2.669617e+07   
4  2025-01-01T09:36:55.755Z  2.647437e+07  2.662856e+07  2.663656e+07   

    clockBias_x  
0 -6.306459e-07  
1 -6.621698e-07  
2 -1.018604e-07  
3  2.311753e-08  
4  5.956487e-07  


In [4]:
def preprocess_gnss_data(data, target_columns=['X', 'Y', 'Z', 'ClockBias_x']):
    """Preprocess GNSS data for time series forecasting"""

    print(f"Available columns in dataset: {data.columns.tolist()}")

    # Map expected columns to actual columns (case-insensitive matching)
    column_mapping = {}
    available_columns = []

    for target_col in target_columns:
        # Find matching column (case-insensitive)
        matching_cols = [col for col in data.columns if col.lower() == target_col.lower()]
        if matching_cols:
            column_mapping[target_col] = matching_cols[0]
            available_columns.append(matching_cols[0])
        else:
            print(f"Warning: Column '{target_col}' not found in data")

    if not available_columns:
        raise ValueError("No matching columns found in the dataset")

    print(f"Using columns: {available_columns}")

    # Select and rename columns
    processed_data = data[available_columns].copy()

    # Rename columns to match expected names
    rename_dict = {v: k for k, v in column_mapping.items()}
    processed_data = processed_data.rename(columns=rename_dict)

    # Handle missing values
    print(f"Data shape before removing NaN: {processed_data.shape}")
    processed_data = processed_data.dropna()
    print(f"Data shape after removing NaN: {processed_data.shape}")

    if processed_data.empty:
        raise ValueError("No data remaining after removing NaN values")

    # Scale the data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_data = scaler.fit_transform(processed_data)

    feature_columns = list(processed_data.columns)

    return scaled_data, scaler, feature_columns

def create_sequences(data, n_timesteps=96):
    """Create input sequences for GAN training"""
    if len(data) <= n_timesteps:
        raise ValueError(f"Dataset too small. Need at least {n_timesteps + 1} samples, got {len(data)}")

    X = []
    for i in range(len(data) - n_timesteps):
        sequence = data[i:(i + n_timesteps)]
        X.append(sequence)

    return np.array(X)

# Preprocess data with the fixed function
try:
    scaled_data, scaler, feature_columns = preprocess_gnss_data(data)
    print(f"Features used: {feature_columns}")
    print(f"Scaled data shape: {scaled_data.shape}")

    # Create sequences
    n_timesteps = 96  # 1 day = 96 intervals of 15 minutes

    # Check if we have enough data
    if len(scaled_data) <= n_timesteps:
        print(f"Warning: Dataset has only {len(scaled_data)} samples.")
        print(f"Reducing n_timesteps from {n_timesteps} to {max(1, len(scaled_data)//2)}")
        n_timesteps = max(1, len(scaled_data)//2)

    sequences = create_sequences(scaled_data, n_timesteps)
    print(f"Total sequences created: {sequences.shape}")

    # Split into training (first 7 days) and test (8th day) data
    n_features = len(feature_columns)

    if len(sequences) > 96:
        train_sequences = sequences[:-96]  # All but last day
        test_input = sequences[-96:-1]     # Last available sequence for prediction
    else:
        # If we don't have 8 days of data, use most for training and few for testing
        split_point = max(1, int(0.9 * len(sequences)))
        train_sequences = sequences[:split_point]
        test_input = sequences[split_point:split_point+1]

    print(f"Training sequences: {train_sequences.shape}")
    print(f"Test input shape: {test_input.shape}")
    print(f"Number of features: {n_features}")
    print(f"Timesteps per sequence: {n_timesteps}")

except Exception as e:
    print(f"Error in data preprocessing: {str(e)}")
    print("Please check your dataset format and content.")


Available columns in dataset: ['epoch', 'x', 'y', 'z', 'clockBias_x']
Using columns: ['x', 'y', 'z', 'clockBias_x']
Data shape before removing NaN: (672, 4)
Data shape after removing NaN: (672, 4)
Features used: ['X', 'Y', 'Z', 'ClockBias_x']
Scaled data shape: (672, 4)
Total sequences created: (576, 96, 4)
Training sequences: (480, 96, 4)
Test input shape: (95, 96, 4)
Number of features: 4
Timesteps per sequence: 96


In [5]:
def build_generator(n_timesteps, n_features, latent_dim=100):
    """Build Generator Network"""
    model = Sequential([
        # Input layer
        Input(shape=(n_timesteps, n_features)),

        # LSTM layers
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64, return_sequences=True),
        Dropout(0.2),

        # Dense layers for generating next sequence
        Dense(32, activation='relu'),
        Dense(n_features, activation='tanh')  # Output next timestep
    ])

    return model

def build_discriminator(n_timesteps, n_features):
    """Build Discriminator Network"""
    model = Sequential([
        # Input layer
        Input(shape=(n_timesteps, n_features)),

        # LSTM layers
        LSTM(64, return_sequences=True),
        Dropout(0.3),
        LSTM(32, return_sequences=False),
        Dropout(0.3),

        # Classification layers
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    return model

# Build models
generator = build_generator(n_timesteps, n_features)
discriminator = build_discriminator(n_timesteps, n_features)

print("Generator Summary:")
generator.summary()
print("\nDiscriminator Summary:")
discriminator.summary()


Generator Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 96, 128)        │        68,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 96, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 96, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 96, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 96, 32)         │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96, 4)          │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,716 (467.64 KB)

 Trainable params: 119,716 (467.64 KB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 96, 64)         │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,625 (119.63 KB)

 Trainable params: 30,625 (119.63 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
class GANLoss:
    """Custom loss functions for GAN training"""
    def __init__(self):
        self.bce_loss = BinaryCrossentropy()
        self.mse_loss = MeanSquaredError()

    def discriminator_loss(self, real_output, fake_output):
        """Discriminator loss: maximize ability to distinguish real vs fake"""
        real_loss = self.bce_loss(tf.ones_like(real_output), real_output)
        fake_loss = self.bce_loss(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    def generator_loss(self, fake_output, real_sequence, generated_sequence, lambda_recon=10.0):
        """Generator loss: fool discriminator + reconstruction accuracy"""
        # Adversarial loss
        adversarial_loss = self.bce_loss(tf.ones_like(fake_output), fake_output)

        # Reconstruction loss
        reconstruction_loss = self.mse_loss(real_sequence, generated_sequence)

        # Combined loss
        total_loss = adversarial_loss + lambda_recon * reconstruction_loss
        return total_loss, adversarial_loss, reconstruction_loss

# Initialize loss functions and optimizers
gan_loss = GANLoss()
generator_optimizer = Adam(2e-4, beta_1=0.5)
discriminator_optimizer = Adam(2e-4, beta_1=0.5)


In [8]:
@tf.function
def train_step(real_sequences):
    """Single training step for GAN"""
    batch_size = tf.shape(real_sequences)[0]

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake sequences
        generated_sequences = generator(real_sequences, training=True)

        # Get discriminator outputs
        real_output = discriminator(real_sequences, training=True)
        fake_output = discriminator(generated_sequences, training=True)

        # Calculate losses
        disc_loss = gan_loss.discriminator_loss(real_output, fake_output)
        gen_loss, adv_loss, recon_loss = gan_loss.generator_loss(
            fake_output, real_sequences, generated_sequences
        )

    # Calculate gradients
    generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Apply gradients
    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss, adv_loss, recon_loss

def train_gan(train_data, epochs=100, batch_size=32):
    """Train the GAN model"""
    dataset = tf.data.Dataset.from_tensor_slices(train_data)
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

    # Training history
    history = {
        'gen_loss': [],
        'disc_loss': [],
        'adv_loss': [],
        'recon_loss': []
    }

    print(f"Starting GAN training for {epochs} epochs...")

    for epoch in range(epochs):
        epoch_gen_loss = 0
        epoch_disc_loss = 0
        epoch_adv_loss = 0
        epoch_recon_loss = 0
        num_batches = 0

        for batch in dataset:
            gen_loss, disc_loss, adv_loss, recon_loss = train_step(batch)

            epoch_gen_loss += gen_loss
            epoch_disc_loss += disc_loss
            epoch_adv_loss += adv_loss
            epoch_recon_loss += recon_loss
            num_batches += 1

        # Average losses for the epoch
        epoch_gen_loss /= num_batches
        epoch_disc_loss /= num_batches
        epoch_adv_loss /= num_batches
        epoch_recon_loss /= num_batches

        # Store history
        history['gen_loss'].append(float(epoch_gen_loss))
        history['disc_loss'].append(float(epoch_disc_loss))
        history['adv_loss'].append(float(epoch_adv_loss))
        history['recon_loss'].append(float(epoch_recon_loss))

        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Gen Loss: {epoch_gen_loss:.4f}")
            print(f"  Disc Loss: {epoch_disc_loss:.4f}")
            print(f"  Adv Loss: {epoch_adv_loss:.4f}")
            print(f"  Recon Loss: {epoch_recon_loss:.4f}")

    return history

# Train the GAN
training_history = train_gan(train_sequences, epochs=50, batch_size=16)


Starting GAN training for 50 epochs...
Epoch 10/50
  Gen Loss: 1.4532
  Disc Loss: 1.3895
  Adv Loss: 0.6853
  Recon Loss: 0.0768
Epoch 20/50
  Gen Loss: 1.4476
  Disc Loss: 1.4430
  Adv Loss: 0.6735
  Recon Loss: 0.0774
Epoch 30/50
  Gen Loss: 1.0928
  Disc Loss: 1.4030
  Adv Loss: 0.7079
  Recon Loss: 0.0385
Epoch 40/50
  Gen Loss: 1.0474
  Disc Loss: 1.3992
  Adv Loss: 0.7050
  Recon Loss: 0.0342
Epoch 50/50
  Gen Loss: 1.0275
  Disc Loss: 1.3922
  Adv Loss: 0.7178
  Recon Loss: 0.0310


In [9]:
def generate_8th_day_predictions(generator, last_sequence, n_steps=96):
    """Generate predictions for the 8th day"""
    predictions = []
    current_sequence = last_sequence.copy()

    for i in range(n_steps):
        # Generate next timestep
        next_step = generator.predict(current_sequence.reshape(1, n_timesteps, n_features), verbose=0)
        predictions.append(next_step[0, -1, :])  # Take the last timestep

        # Update sequence for next prediction
        current_sequence = np.roll(current_sequence, -1, axis=0)
        current_sequence[-1] = next_step[0, -1, :]

    return np.array(predictions)

# Generate 8th day predictions
if len(test_input) > 0:
    last_sequence = test_input[0]  # Use the last available sequence
    day_8_predictions = generate_8th_day_predictions(generator, last_sequence)

    # Inverse transform predictions back to original scale
    day_8_predictions_original = scaler.inverse_transform(day_8_predictions)

    print(f"Generated predictions shape: {day_8_predictions_original.shape}")
else:
    print("No test input available for prediction")


Generated predictions shape: (96, 4)


In [1]:
def evaluate_multi_horizon_predictions(predictions, feature_names):
    """Evaluate predictions at multiple time horizons"""
    horizons = [1, 2, 4, 8, 16, 32, 64, 96]  # 15min, 30min, 1hr, 2hr, 4hr, 8hr, 16hr, 24hr
    horizon_names = ['15min', '30min', '1hr', '2hr', '4hr', '8hr', '16hr', '24hr']

    print("Multi-Horizon Evaluation Results:")
    print("-" * 50)

    for i, (horizon, name) in enumerate(zip(horizons, horizon_names)):
        if horizon <= len(predictions):
            pred_slice = predictions[:horizon]
            print(f"{name:>8} | Samples: {len(pred_slice):3d} | "
                  f"Mean: {np.mean(pred_slice, axis=0)} | "
                  f"Std: {np.std(pred_slice, axis=0)}")

def create_comprehensive_visualizations(predictions, feature_names, training_history=None):
    """Create comprehensive visualizations including histograms and scatter plots"""

    n_features = len(feature_names)
    time_points = np.arange(len(predictions)) * 15  # 15-minute intervals

    # 1. Time series plots for each feature
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('8th Day GNSS Error Predictions - Time Series', fontsize=16, y=0.95)

    for i, (feature, ax) in enumerate(zip(feature_names, axes.flat)):
        if i < n_features:
            ax.plot(time_points, predictions[:, i], 'b-', linewidth=2,
                   label='Predicted', alpha=0.8)
            ax.set_title(f'{feature} Error Prediction Over Time')
            ax.set_xlabel('Time (minutes)')
            ax.set_ylabel('Error Value')
            ax.grid(True, alpha=0.3)
            ax.legend()

            # Add statistics text
            mean_val = np.mean(predictions[:, i])
            std_val = np.std(predictions[:, i])
            ax.text(0.02, 0.98, f'Mean: {mean_val:.4f}\nStd: {std_val:.4f}',
                   transform=ax.transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        else:
            ax.axis('off')  # Hide unused subplots

    plt.tight_layout()
    plt.show()

    # 2. Histogram of predictions for each feature
    fig, axes = plt.subplots(1, n_features, figsize=(4*n_features, 5))
    fig.suptitle('Distribution of Predicted Values - Histograms', fontsize=16)

    if n_features == 1:
        axes = [axes]  # Make it iterable for single feature

    colors = ['skyblue', 'lightgreen', 'salmon', 'gold']
    for i, (feature, ax) in enumerate(zip(feature_names, axes)):
        ax.hist(predictions[:, i], bins=25, alpha=0.7, color=colors[i % len(colors)],
                edgecolor='black', linewidth=0.5)
        ax.set_title(f'{feature} Distribution')
        ax.set_xlabel('Predicted Value')
        ax.set_ylabel('Frequency')
        ax.grid(True, alpha=0.3)

        # Add statistical information
        mean_val = np.mean(predictions[:, i])
        std_val = np.std(predictions[:, i])
        ax.axvline(mean_val, color='red', linestyle='--', linewidth=2,
                  label=f'Mean: {mean_val:.4f}')
        ax.axvline(mean_val + std_val, color='orange', linestyle=':',
                  label=f'+1σ: {mean_val+std_val:.4f}')
        ax.axvline(mean_val - std_val, color='orange', linestyle=':',
                  label=f'-1σ: {mean_val-std_val:.4f}')
        ax.legend(fontsize=8)

    plt.tight_layout()
    plt.show()

    # 3. Scatter plots - Feature correlations
    if n_features > 1:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Feature Correlation Scatter Plots', fontsize=16)

        plot_idx = 0
        for i in range(n_features):
            for j in range(i+1, n_features):
                if plot_idx < 6:  # Maximum 6 scatter plots
                    row, col = divmod(plot_idx, 3)
                    ax = axes[row, col]

                    ax.scatter(predictions[:, i], predictions[:, j],
                             alpha=0.6, c=time_points, cmap='viridis', s=30)
                    ax.set_xlabel(f'{feature_names[i]} Predictions')
                    ax.set_ylabel(f'{feature_names[j]} Predictions')
                    ax.set_title(f'{feature_names[i]} vs {feature_names[j]}')
                    ax.grid(True, alpha=0.3)

                    # Add correlation coefficient
                    corr = np.corrcoef(predictions[:, i], predictions[:, j])[0, 1]
                    ax.text(0.02, 0.98, f'Corr: {corr:.3f}',
                           transform=ax.transAxes, verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

                    plot_idx += 1

        # Hide unused subplots
        for idx in range(plot_idx, 6):
            row, col = divmod(idx, 3)
            axes[row, col].axis('off')

        plt.tight_layout()
        plt.show()

    # 4. Prediction residuals analysis (if we had true values, this would be error analysis)
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('Prediction Residuals and Statistical Analysis', fontsize=16)

    # Residuals from mean (since we don't have true values)
    for i, (feature, ax) in enumerate(zip(feature_names, axes.flat)):
        if i < n_features:
            mean_pred = np.mean(predictions[:, i])
            residuals = predictions[:, i] - mean_pred

            ax.hist(residuals, bins=20, alpha=0.7, color=colors[i % len(colors)],
                   edgecolor='black')
            ax.set_title(f'{feature} Residuals from Mean')
            ax.set_xlabel('Residual Value')
            ax.set_ylabel('Frequency')
            ax.grid(True, alpha=0.3)

            # Test normality
            from scipy import stats
            _, p_value = stats.shapiro(residuals)
            normality_text = "Normal" if p_value > 0.05 else "Not Normal"
            ax.text(0.02, 0.98, f'Shapiro p-value: {p_value:.4f}\n{normality_text}',
                   transform=ax.transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()

    # 5. Training history (if available)
    if training_history is not None:
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle('GAN Training History', fontsize=16)

        epochs = range(1, len(training_history['gen_loss']) + 1)

        axes[0, 0].plot(epochs, training_history['gen_loss'], 'b-', label='Generator Loss', linewidth=2)
        axes[0, 0].plot(epochs, training_history['disc_loss'], 'r-', label='Discriminator Loss', linewidth=2)
        axes[0, 0].set_title('Generator vs Discriminator Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        axes[0, 1].plot(epochs, training_history['adv_loss'], 'orange', linewidth=2)
        axes[0, 1].set_title('Adversarial Loss')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].grid(True, alpha=0.3)

        axes[1, 0].plot(epochs, training_history['recon_loss'], 'green', linewidth=2)
        axes[1, 0].set_title('Reconstruction Loss')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Loss')
        axes[1, 0].grid(True, alpha=0.3)

        # Combined loss trends
        axes[1, 1].plot(epochs, training_history['gen_loss'], 'b-', alpha=0.7, label='Generator')
        axes[1, 1].plot(epochs, training_history['adv_loss'], 'orange', alpha=0.7, label='Adversarial')
        axes[1, 1].plot(epochs, training_history['recon_loss'], 'green', alpha=0.7, label='Reconstruction')
        axes[1, 1].set_title('All Loss Components')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    # 6. Statistical summary table
    print("\nStatistical Summary of Predictions:")
    print("=" * 60)

    stats_df = pd.DataFrame({
        'Feature': feature_names,
        'Mean': [np.mean(predictions[:, i]) for i in range(n_features)],
        'Std': [np.std(predictions[:, i]) for i in range(n_features)],
        'Min': [np.min(predictions[:, i]) for i in range(n_features)],
        'Max': [np.max(predictions[:, i]) for i in range(n_features)],
        'Range': [np.ptp(predictions[:, i]) for i in range(n_features)]
    })

    print(stats_df.round(6))

# Enhanced evaluation and visualization
if 'day_8_predictions_original' in locals():
    evaluate_multi_horizon_predictions(day_8_predictions_original, feature_columns)

    # Check if training_history exists
    history = training_history if 'training_history' in locals() else None

    create_comprehensive_visualizations(day_8_predictions_original, feature_columns, history)

    # Save predictions
    predictions_df = pd.DataFrame(day_8_predictions_original, columns=feature_columns)
    predictions_df.to_csv('day_8_predictions.csv', index=False)
    print("\nPredictions saved to 'day_8_predictions.csv'")
else:
    print("No predictions available for evaluation")


No predictions available for evaluation
